# **FEATURE ENGINEERING AND DATA PREPARATION** 

In [1]:
import warnings
warnings.simplefilter(action = 'ignore')
import numpy as np 
import pandas as pd 
import gc

In [2]:
targets = pd.read_hdf("./Data/train_cite_targets.h5")
meta_df = pd.read_csv('./Data/metadata.csv', index_col = 'cell_id')
meta_df = meta_df[meta_df.technology == 'citeseq'].drop(columns = 'technology')
meta_df['gender'] = meta_df.donor.apply(lambda x : 1 if x != 13176 else 0)
meta_df

,day,donor,cell_type,gender
cell_id,,,,
c2150f55becb,2,27678,HSC,1
65b7edf8a4da,2,27678,HSC,1
c1b26cb1057b,2,27678,EryP,1
917168fa6f83,2,27678,NeuP,1
2b29feeca86d,2,27678,EryP,1
...,...,...,...,...
a9b4d99f1f50,7,31800,HSC,1
0e2c1d0782af,7,31800,HSC,1
a3cbc5aa0ec3,7,31800,MkP,1


In [3]:
def fix_test():        # Test Data Had Some Issue Which Was Later Fixed By The Publisher
    global test, meta_df
    extra_columns = ['ENSG00000255655_AC007570.1','ENSG00000213237_AC008155.1','ENSG00000273398_AC017083.3','ENSG00000233849_AC022201.2','ENSG00000231401_AC023481.1','ENSG00000263126_AC040162.3','ENSG00000285876_AC093912.1','ENSG00000275585_AC241377.2','ENSG00000272175_AL050343.2','ENSG00000232290_AL133260.2','ENSG00000267559_AL158154.2','ENSG00000273481_AL391069.4','ENSG00000161609_CCDC155','ENSG00000150394_CDH8','ENSG00000233932_CTXN2','ENSG00000198077_CYP2A7','ENSG00000220871_DNAJC19P6','ENSG00000267336_EIF4A2P1','ENSG00000251254_GTF2F2P1','ENSG00000225978_HAR1A','ENSG00000131097_HIGD1B','ENSG00000211897_IGHG3','ENSG00000233080_LINC01399','ENSG00000236849_LINC01474','ENSG00000269904_MAP2K4P1','ENSG00000266698_MIR3945','ENSG00000144227_NXPH2','ENSG00000182057_OGFRP1','ENSG00000251107_PDCD5P2','ENSG00000226964_RHEBP2','ENSG00000214988_RPL7AP26','ENSG00000248873_SERBP1P6','ENSG00000212588_SNORA26','ENSG00000166984_TCP10L2','ENSG00000226051_ZNF503-AS1']
    meta_fix = pd.read_csv("./Data/metadata_cite_day_2_donor_27678.csv", index_col = 'cell_id')
    test_fix = pd.read_hdf("./Data/test_cite_inputs_day_2_donor_27678.h5")
    test_fix.drop(columns = extra_columns, inplace = True)
    filter_out = meta_df[(meta_df.day == 2) & (meta_df.donor == 27678)].index
    test.drop(index = filter_out, inplace = True)
    test = pd.concat([test, test_fix], axis = 0)
    meta_df.drop(index = filter_out, inplace = True)
    meta_df = pd.concat([meta_df, meta_fix], axis = 0)
    
    del extra_columns, meta_fix, test_fix, filter_out
    gc.collect()

In [4]:
train = pd.read_hdf("./Data/train_cite_inputs.h5")
meta_train = meta_df.reindex(train.index)
train = pd.concat([train, meta_train[['day', 'donor', 'gender']]], axis = 1)

# Remove Useless Data And Adding Important Columns
notimp_cols = list(train.columns[(train == 0).all().values])
imp_cols = [_ for _ in train.columns if _[_.find('_') + 1 : ] in targets.columns] + ['day', 'donor', 'gender']
print(f"{len(notimp_cols)} Null Columns and {len(imp_cols)} Important Columns")

from sklearn.decomposition import TruncatedSVD as tSVD
def transform(data, dimension):
    global notimp_cols, imp_cols
    print("Shape Of Data Before Processing ",data.shape)
    data.drop(columns = notimp_cols, inplace = True)
    print('- useless columns dropped')
    
    data_imp_cols = data[imp_cols].to_numpy()
    data.drop(columns = ['day', 'donor', 'gender'], inplace = True)
    
    print('- important columns saved')
    data = data - data.median() 
    data = data.apply(lambda row: row / row.std(), axis = 1)   
    print('- standardized the data')
    data = data.to_numpy()
    # Reducing them to lower dimensions using PCA
    pca = tSVD(n_components = dimension, random_state = 42)
    data = pca.fit_transform(data)
    evr = pca.explained_variance_ratio_
    print(f"- transformed to {dimension} dimensions : explaining {str(sum(evr)*100)[:5]} % of data")
    data = np.hstack([data, data_imp_cols])
    print('- stacked the important columns')
    print("Shape Of Data After Removal ",data.shape)
    print("-"* 65)
    print()
    return data

449 Null Columns and 37 Important Columns


In [5]:
new_dim = 100
print("-"*25, "For Train Data", "-"*25)
train = transform(train, new_dim)

test = pd.read_hdf("./Data/test_cite_inputs.h5")
fix_test()
meta_test = meta_df.reindex(test.index)
test = pd.concat([test, meta_test[['day', 'donor', 'gender']]], axis = 1)

print("-"*22, "For Donor Test Data", "-"*22)
meta_donor_test = meta_test[(meta_test.day != 7) & (meta_test.donor == 27678)]
donor_test = test.reindex(meta_donor_test.index)
donor_test = transform(donor_test, new_dim)

print("-"*22, "For Day Test Data", "-"*22)
meta_day_test = meta_test[meta_test.day == 7]
day_test = test.reindex(meta_day_test.index)
del test     #useless data now, deleting to free up space
gc.collect()
day_test = transform(day_test, new_dim)

targets = targets.values

------------------------- For Train Data -------------------------
Shape Of Data Before Processing  (70988, 22053)
- useless columns dropped
- important columns saved
- standardized the data
- transformed to 100 dimensions : explaining 12.90 % of data
- stacked the important columns
Shape Of Data After Removal  (70988, 137)
-----------------------------------------------------------------

---------------------- For Donor Test Data ----------------------
Shape Of Data Before Processing  (21336, 22053)
- useless columns dropped
- important columns saved
- standardized the data
- transformed to 100 dimensions : explaining 13.21 % of data
- stacked the important columns
Shape Of Data After Removal  (21336, 137)
-----------------------------------------------------------------

---------------------- For Day Test Data ----------------------
Shape Of Data Before Processing  (26867, 22053)
- useless columns dropped
- important columns saved
- standardized the data
- transformed to 100 dimens

In [6]:
del notimp_cols, imp_cols, meta_df
gc.collect()

0

In [7]:
# Saving The Clean Data
np.save('train', train)
np.save('donor_test', donor_test)
np.save('day_test', day_test)
np.save('targets', targets)

--------

# **MODEL BUILDING** 

In [8]:
# Trun On The GPU And Run This (Not required if running locally)
import warnings
warnings.simplefilter(action = 'ignore')
import numpy as np 
import pandas as pd 
import gc

# Load The Data
train = np.load("train.npy")
targets = np.load("targets.npy")
donor_test = np.load("train.npy")
day_test = np.load("train.npy")

In [9]:
# Choosing features important for training each target column
corr_col_dict = {}
def corr_cols():
    global corr_col_dict
    for out in range(targets.shape[1]):
        correlations = []
        for inp in range(train.shape[1]):
            correlations.append(np.corrcoef(train[:, inp], targets[:, out])[0, 1])
        correlated_features = [correlations.index(i) for i in correlations if abs(i) > np.percentile(list(map(abs, correlations)), 75)]
        corr_col_dict[out] = correlated_features
corr_cols()

In [10]:
def correlation_score(y_true, y_hat):
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_hat) == pd.DataFrame: y_hat = y_hat.values
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_hat[i])[1, 0]
    return corrsum / len(y_true)

In [11]:
!pip uninstall lightgbm -y
!pip install lightgbm --config-settings=cmake.define.USE_GPU=ON
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor

# Params For GroupKfold By Donor
xgb_params = {
    'n_estimators' : 300,
    'tree_method' : 'gpu_hist',
    'predictor' : 'gpu_predictor',
    'gpu_id' : 0
}
lgbm_params = {
     'n_estimators' : 300,
     'learning_rate': 0.1, 
     'max_depth': 10, 
     'num_leaves': 200,
     'min_child_samples': 250,
     'colsample_bytree': 0.8, 
     'subsample': 0.6, 
     "seed" : 1,
     "device" : "gpu",
     "verbose" : -1
}
cat_params = {
    "iterations" : 300,
    "depth" : 6, 
    "learning_rate" : 0.1,
    "loss_function" : "RMSE", 
    "verbose" : 0
}

# Initialize Individual Regressors
xgb_regressor = XGBRegressor(**xgb_params)
lgbm_regressor = LGBMRegressor(**lgbm_params)
catboost_regressor = CatBoostRegressor(**cat_params)
ensemble_regressor = VotingRegressor(estimators=[
    ('xgb', xgb_regressor),
    ('catboost', catboost_regressor),
    ('lgbm', lgbm_regressor)
])

Found existing installation: lightgbm 4.1.0
Uninstalling lightgbm-4.1.0:
  Successfully uninstalled lightgbm-4.1.0
  Using cached lightgbm-4.1.0-py3-none-manylinux_2_28_x86_64.whl (3.1 MB)


In [12]:
# KFold Grouped By Donor
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from colorama import Fore, Style

# for the purpose of fancy visuals
from tqdm import tqdm as progress_bar
import time

y_cols = targets.shape[1]
groups = train[ : , -2]

kf = GroupKFold(n_splits = 3)
score_list = []
for fold, (idx_tr, idx_va) in enumerate(kf.split(train, groups = groups)):
    X_tr = train[idx_tr]
    y_tr = targets[:,: y_cols][idx_tr]
    X_va = train[idx_va]
    y_va = targets[:,: y_cols][idx_va]

    pred_list = []
    with progress_bar(total = y_cols, bar_format = '{desc} : |{bar}|{percentage:3.0f}%  {n_fmt}/{total_fmt}', desc = f'Training Fold {fold} with Donors {list(map(int, set(train[idx_tr][: , -2])))} for Donor {list(map(int, set(train[idx_va][: , -2])))}', unit = 'iteration') as pbar:
        for i in range(y_cols):
            X_tr_corr = X_tr[:, corr_col_dict[i]]
            X_va_corr = X_va[:, corr_col_dict[i]]
            
            ensemble_regressor.fit(X_tr_corr, y_tr[:,i].copy())
            pred_list.append(ensemble_regressor.predict(X_va_corr))
                
            time.sleep(0.1)
            pbar.update(1)
    y_hat = np.column_stack(pred_list) # concatenate the 140 predictions
    del pred_list, X_tr, y_tr, X_tr_corr, X_va_corr
    gc.collect()

    # We validate the model (mse and correlation over all 140 columns)
    mse = mean_squared_error(y_va, y_hat)
    corrscore = correlation_score(y_va, y_hat)

    del X_va, y_va
    gc.collect()

    print(f"Fold {fold} {train.shape[1]:4}: mse = {mse:.2f}, corr =  {corrscore:.2f}")
    score_list.append((mse, corrscore))

# Averaging Scores From All The Folds
if len(score_list) > 1:
    result_df = pd.DataFrame(score_list, columns=['mse', 'corrscore'])
    print(Fore.YELLOW + f"Average MSE : {result_df.mse.mean():.2f}, Average Correlation : {result_df.corrscore.mean():.2f}"+ Style.RESET_ALL)


Training Fold 0 with Donors [13176, 32606] for Donor [31800] : |██████████|100%  140/140


Fold 0  137: mse = 2.39, corr =  0.88


Training Fold 1 with Donors [13176, 31800] for Donor [32606] : |██████████|100%  140/140


Fold 1  137: mse = 2.37, corr =  0.89


Training Fold 2 with Donors [31800, 32606] for Donor [13176] : |██████████|100%  140/140


Fold 2  137: mse = 4.06, corr =  0.88
Average MSE : 2.94, Average Correlation : 0.88


In [13]:
# KFold Grouped By Day
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from colorama import Fore, Style

# for the purpose of fancy visuals
from tqdm import tqdm as progress_bar
import time

y_cols = targets.shape[1]
groups = train[ : , -3]

kf = GroupKFold(n_splits = 3)
score_list = []
for fold, (idx_tr, idx_va) in enumerate(kf.split(train, groups = groups)):
    X_tr = train[idx_tr]
    y_tr = targets[:,: y_cols][idx_tr]
    X_va = train[idx_va]
    y_va = targets[:,: y_cols][idx_va]

    pred_list = []
    with progress_bar(total = y_cols, bar_format = '{desc} : |{bar}|{percentage:3.0f}%  {n_fmt}/{total_fmt}', desc = f'Training Fold {fold} with Days {list(map(int, set(train[idx_tr][: , -3])))} for Day {list(map(int, set(train[idx_va][: , -3])))}', unit = 'iteration') as pbar:
        for i in range(y_cols):
            X_tr_corr = X_tr[:, corr_col_dict[i]]
            X_va_corr = X_va[:, corr_col_dict[i]]
            
            ensemble_regressor.fit(X_tr_corr, y_tr[:,i].copy())
            pred_list.append(ensemble_regressor.predict(X_va_corr))
                
            time.sleep(0.1)
            pbar.update(1)
    y_hat = np.column_stack(pred_list) # concatenate the 140 predictions
    del pred_list, X_tr, y_tr, X_tr_corr, X_va_corr
    gc.collect()

    # We validate the model (mse and correlation over all 140 columns)
    mse = mean_squared_error(y_va, y_hat)
    corrscore = correlation_score(y_va, y_hat)

    del X_va, y_va
    gc.collect()

    print(f"Fold {fold} {train.shape[1]:4}: mse = {mse:.2f}, corr =  {corrscore:.2f}")
    score_list.append((mse, corrscore))

# Averaging Scores From All The Folds
if len(score_list) > 1:
    result_df = pd.DataFrame(score_list, columns=['mse', 'corrscore'])
    print(Fore.YELLOW + f"Average MSE : {result_df.mse.mean():.2f}, Average Correlation : {result_df.corrscore.mean():.2f}"+ Style.RESET_ALL)


Training Fold 0 with Days [2, 3] for Day [4] : |██████████|100%  140/140


Fold 0  137: mse = 2.64, corr =  0.88


Training Fold 1 with Days [3, 4] for Day [2] : |██████████|100%  140/140


Fold 1  137: mse = 2.85, corr =  0.89


Training Fold 2 with Days [2, 4] for Day [3] : |██████████|100%  140/140


Fold 2  137: mse = 2.97, corr =  0.89
Average MSE : 2.82, Average Correlation : 0.89


RETRAINING ON WHOLE DATA

In [15]:
pred_list = []
with progress_bar(total = y_cols, bar_format = '{desc} : |{bar}|{percentage:3.0f}%  {n_fmt}/{total_fmt}', desc = 'Retraining On Whole Data ', unit='iteration') as pbar:
    for i in range(y_cols):
        pt_corr = train[:, corr_col_dict[i]]
        ensemble_regressor.fit(pt_corr, targets[:,i].copy())
        pred_list.append(ensemble_regressor.predict(pt_corr))
        time.sleep(0.1)
        pbar.update(1)
y_hat = np.column_stack(pred_list) # concatenate the 140 predictions
mse = mean_squared_error(targets, y_hat)
corrscore = correlation_score(targets, y_hat)
print(f'MSE On Whole Data is {mse : .2f} and Correlation Score is {corrscore : .2f}')

Retraining On Whole Data  : |██████████|100%  140/140


MSE On Whole Data is  1.50 and Correlation Score is  0.93


In [16]:
pred_list = []
with progress_bar(total = y_cols, bar_format = '{desc} : |{bar}|{percentage:3.0f}%  {n_fmt}/{total_fmt}', desc = 'Testing On Data With Donor 4 ', unit='iteration') as pbar:
    for i in range(y_cols):
        pt_corr = donor_test[:, corr_col_dict[i]]
        pred_list.append(ensemble_regressor.predict(pt_corr))
        time.sleep(0.1)
        pbar.update(1)
y_hat = np.column_stack(pred_list) # concatenate the 140 predictions
print(y_hat)

pred_list = []
with progress_bar(total = y_cols, bar_format = '{desc} : |{bar}|{percentage:3.0f}%  {n_fmt}/{total_fmt}', desc = 'Testing On Data From A Different Day ', unit='iteration') as pbar:
    for i in range(y_cols):
        pt_corr = day_test[:, corr_col_dict[i]]
        pred_list.append(ensemble_regressor.predict(pt_corr))
        time.sleep(0.1)
        pbar.update(1)
y_hat = np.column_stack(pred_list) # concatenate the 140 predictions
print(y_hat)

Testing On Data With Donor 4  : |██████████|100%  140/140


[[1.04630607 2.00784029 2.2451049  ... 1.65586986 2.40053819 1.19232374]
 [1.79529695 1.56196776 1.84762049 ... 1.43062198 1.87287806 1.28844676]
 [1.10319961 1.80775265 1.77694262 ... 1.70479396 1.82852748 2.08570162]
 ...
 [2.610289   2.77716355 2.3372659  ... 2.38049246 1.58823294 3.75252575]
 [2.35452687 2.98882973 4.14508958 ... 2.62446883 3.12422974 3.41985365]
 [2.09925628 3.12575942 2.24207375 ... 1.98744017 2.75329372 3.54321394]]


Testing On Data From A Different Day  : |██████████|100%  140/140

[[1.04630607 2.00784029 2.2451049  ... 1.65586986 2.40053819 1.19232374]
 [1.79529695 1.56196776 1.84762049 ... 1.43062198 1.87287806 1.28844676]
 [1.10319961 1.80775265 1.77694262 ... 1.70479396 1.82852748 2.08570162]
 ...
 [2.610289   2.77716355 2.3372659  ... 2.38049246 1.58823294 3.75252575]
 [2.35452687 2.98882973 4.14508958 ... 2.62446883 3.12422974 3.41985365]
 [2.09925628 3.12575942 2.24207375 ... 1.98744017 2.75329372 3.54321394]]


-----------------